# Analizar el Dataset de ventas en Supermercados.

Responder las siguientes preguntas:

* Cual es la marca mas vendida segun sus cantidades?
* que tipo de cliente tiene un precio unitario promedio mas alto
* que marca tiene el precio unitario promedio mas alto
* que genero lleva gasta mas importe segun sus montos por Compra?

# SPARK SQL

In [0]:
#Importamos los archivos y los cargamos en Dataframe

df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/cristian1701@gmail.com/supermarket_sales___Sheet1.csv")



In [0]:

# Creamos una Vista para poder consultar los datos mediante SPARKSQL

df1.createOrReplaceTempView("vista")

spark.sql("SELECT * FROM vista").show(truncate=False)

+-----------+------+---------+-------------+------+----------------------+----------+--------+-------+--------+---------+-----+-----------+------+-----------------------+------------+------+
|Invoice ID |Branch|City     |Customer type|Gender|Product line          |Unit price|Quantity|Tax 5% |Total   |Date     |Time |Payment    |cogs  |gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+----------------------+----------+--------+-------+--------+---------+-----+-----------+------+-----------------------+------------+------+
|750-67-8428|A     |Yangon   |Member       |Female|Health and beauty     |74.69     |7       |26.1415|548.9715|1/5/2019 |13:08|Ewallet    |522.83|4.761904762            |26.1415     |9.1   |
|226-31-3081|C     |Naypyitaw|Normal       |Female|Electronic accessories|15.28     |5       |3.82   |80.22   |3/8/2019 |10:29|Cash       |76.4  |4.761904762            |3.82        |9.6   |
|631-41-3108|A     |Yangon   |Normal       |M

In [0]:
# Marca mas Vendida 

spark.sql(""" SELECT 
                 Branch
                 ,SUM(Quantity) AS Cantidad 
              FROM 
                  vista 
               GROUP BY
                   Branch 
               ORDER BY
                   Cantidad 
           """).show(truncate=False)

+------+--------+
|Branch|Cantidad|
+------+--------+
|B     |1820.0  |
|C     |1831.0  |
|A     |1859.0  |
+------+--------+



In [0]:
# Que tipo de cliente tiene un precio unitario promedio mas alto
spark.sql(""" SELECT
                   `Customer type`
                  ,AVG(`Unit price`) Precio_Promedio
               FROM
                   vista 
               GROUP BY
                   `Customer type`
            ORDER BY 
                Precio_Promedio DESC
           """).show(truncate=False)

+-------------+-----------------+
|Customer type|Precio_Promedio  |
+-------------+-----------------+
|Member       |56.20698602794411|
|Normal       |55.13513026052104|
+-------------+-----------------+



In [0]:
# Que marca tiene el precio unitario promedio mas alto
spark.sql(""" SELECT 
                 Branch
                  ,AVG(`Unit price`) Precio_Promedio
               FROM
                   Vista 
               GROUP BY
                   Branch
            ORDER BY 
                Precio_Promedio DESC
                
          """         ).show(truncate=False)


+------+------------------+
|Branch|Precio_Promedio   |
+------+------------------+
|C     |56.609024390243896|
|B     |55.65927710843374 |
|A     |54.78085294117646 |
+------+------------------+



In [0]:
# que genero lleva gastado mas importe segun sus montos por Compra?

spark.sql(""" SELECT 
                  Gender
                  ,SUM(Total) Importe_Total
               FROM
                   Vista 
               GROUP BY
                   Gender
            ORDER BY 
                Importe_Total DESC
                
          """         ).show(truncate=False)

+------+------------------+
|Gender|Importe_Total     |
+------+------------------+
|Female|167882.92500000002|
|Male  |155083.82400000014|
+------+------------------+



# PANDAS

In [0]:
df_PD = df1.toPandas()

In [0]:
df_PD.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,3/8/2019,10:29,Cash,76.4,4.761904762,3.82,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761904762,30.2085,5.3


In [0]:
# Marca mas Vendida 

# Convertimos a Numerica

df_PD['Quantity'] = pd.to_numeric(df_PD['Quantity'])

# Sumamos y agrupamos

cuenta = df_PD.groupby('Branch')['Quantity'].sum()

# imprimir la cuenta
print(cuenta)

Branch
A    1859
B    1820
C    1831
Name: Quantity, dtype: int64


In [0]:
# que tipo de cliente tiene un precio unitario promedio mas alto

# Convertimos a Numerica

df_PD['Unit price'] = pd.to_numeric(df_PD['Unit price'])

# Sumamos y agrupamos

Tipo_Cliente = df_PD.groupby('Customer type')['Unit price'].sum()

# imprimir la cuenta
print(Tipo_Cliente)

Customer type
Member    28159.70
Normal    27512.43
Name: Unit price, dtype: float64


In [0]:
# Que marca tiene el precio unitario promedio mas alto

# Promedio y agrupamos

Marca = df_PD.groupby('Customer type')['Unit price'].mean()

# imprimir la cuenta
print(Marca)

Customer type
Member    56.206986
Normal    55.135130
Name: Unit price, dtype: float64


In [0]:
# Que genero lleva gasta mas importe segun sus montos por Compra?
# Convertimos a Numerica
df_PD['Total'] = pd.to_numeric(df_PD['Total'])



Genero = df_PD.groupby('Gender')['Total'].sum()

# imprimir la cuenta
print(Genero)

Gender
Female    167882.925
Male      155083.824
Name: Total, dtype: float64
